In [1]:
import os
import sys
import re
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), "src\\Features")))
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), "src\\Model")))
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), "src\\Data")))

In [3]:
# sys.path

In [4]:
import LoadData as LD
import DataPreProcessing as dp
import Models

# Datapreprocessing

In [5]:
processed_file_path = "../Data/Process/sample_preprocessed_data.xlsx"
if not os.path.exists(processed_file_path):
    print("inside loop")
    path = '../Data/Raw/IMDB Dataset sample.csv'
    df = LD.load_data(path)
    df['review'] = df['review'].apply(lambda x: re.sub(r'<br /><br />', '', x))
    df['review'] = df['review'].apply(lambda x: re.sub(r'\'', '', x))
    df['cleaned_review'] = df['review'].apply(lambda x: dp.preprocess_text(x))
    df.to_excel(processed_file_path, index=False)

# vectorization and modeling

In [6]:
df = pd.read_excel("../Data/Process/sample_preprocessed_data.xlsx")
vectorizer_dict = [{'vectorizer':'count', 'review_column': 'cleaned_review','pretrained':None},
              {'vectorizer':'tf-idf', 'review_column': 'cleaned_review','pretrained':None},
              {'vectorizer':'word2vec', 'review_column': 'cleaned_review','pretrained':False},
              {'vectorizer':'fasttext', 'review_column': 'cleaned_review','pretrained':False},
              {'vectorizer':'word2vec', 'review_column': 'cleaned_review','pretrained':True},
              {'vectorizer':'fasttext', 'review_column': 'cleaned_review','pretrained':True},
                   {'vectorizer':'word2vec', 'review_column': 'review','pretrained':False},
              {'vectorizer':'fasttext', 'review_column': 'review','pretrained':False},
              {'vectorizer':'word2vec', 'review_column': 'review','pretrained':True},
              {'vectorizer':'fasttext', 'review_column': 'review','pretrained':True},
                  ]

models = {
'Logistic Regression',
'Multinomial Naive Bayes',
'Decision Tree',
'Random Forest',
'SVC',
'KNN'
}

In [ ]:
import time
label_encoder = LabelEncoder()
df['sentiment_encoded'] = label_encoder.fit_transform(df['sentiment'])
df_summary = pd.DataFrame(columns=['vectorizer', 'model_name', 'accuracy', 'precision', 'recall', 'f1'])
index=0
t = time.time()
for dic in vectorizer_dict:
    df_ = dp.vectorization(df, vectorizer=dic['vectorizer'], review_column=dic['review_column'], pretrained=dic['pretrained'])
    (X_train, X_test, y_train, y_test) = Models.train_test_data_split(df_)
    for model_name in models:
        print(model_name, dic)
        Models.fit_models(X_train, y_train, model_name)
        accuracy, precision, recall, f1 = Models.predict_models_summary(X_test, y_test, model_name)
        df_summary.loc[index] = [dic['vectorizer'], model_name, accuracy, precision, recall, f1]
        index+=1
        print(index)
print("Done in:",time.time()-t)

SVC {'vectorizer': 'count', 'review_column': 'cleaned_review', 'pretrained': None}


In [ ]:
df_summary